In [4]:
boids

[{'x': 408.6414101146774,
  'y': 237.15993789140734,
  'dx': 5.388072732137723,
  'dy': -13.571754741702286,
  'history': [(229.1043586019997, 8.340728269364757),
   (214.94595371843462, 10.540733700568587),
   (200.71352267444908, 13.95484490991022),
   (186.4188867365342, 18.561688675644255),
   (173.195631347173, 24.32603607366289),
   (160.96837333112148, 30.415973007843043),
   (149.6703407843835, 36.94572707507704),
   (139.2183708309008, 43.97048289835263),
   (129.51802514400637, 51.483402921248185),
   (120.8359605676606, 60.30852742335205),
   (113.15861658939015, 70.42793480723921),
   (106.98472423266935, 81.15653540079889),
   (102.01711554940988, 92.45101432782403),
   (97.97074901071679, 104.19049637523557),
   (94.92133324503801, 117.24696806311465),
   (92.87204242894016, 131.59078910873586),
   (92.67879460373152, 146.99537324677323),
   (94.21829720979517, 162.9856680049843),
   (97.33973681086515, 178.83489337262594),
   (101.86729930884339, 194.41420415272998),
   

In [6]:
import math
import random
import pygame
import sys

# Size of canvas. These get updated to fill the whole screen.
width = 1000
height = 1000

numBoids = 100
visualRange = 75

boids = []
 
def initBoids():
    global boids
    boids = []
    for i in range(numBoids):
        boids.append({
            'x': random.random() * width,
            'y': random.random() * height,
            'dx': random.random() * 10 - 5,
            'dy': random.random() * 10 - 5,
            'history': [],
        })

def distance(boid1, boid2):
    return math.sqrt((boid1['x'] - boid2['x'])**2 + (boid1['y'] - boid2['y'])**2)

def nClosestBoids(boid, n):
    sorted_boids = sorted(boids, key=lambda other_boid: distance(boid, other_boid))
    return sorted_boids[1:n+1]

def sizeCanvas():
    global width, height
    size = (width, height)
    return pygame.display.set_mode(size)

def keepWithinBounds(boid):
    margin = 200
    turnFactor = 1

    if boid['x'] < margin:
        boid['dx'] += turnFactor
    if boid['x'] > width - margin:
        boid['dx'] -= turnFactor
    if boid['y'] < margin:
        boid['dy'] += turnFactor
    if boid['y'] > height - margin:
        boid['dy'] -= turnFactor

def flyTowardsCenter(boid):
    centeringFactor = 0.005

    centerX = 0
    centerY = 0
    numNeighbors = 0

    for otherBoid in boids:
        if distance(boid, otherBoid) < visualRange:
            centerX += otherBoid['x']
            centerY += otherBoid['y']
            numNeighbors += 1

    if numNeighbors:
        centerX /= numNeighbors
        centerY /= numNeighbors

        boid['dx'] += (centerX - boid['x']) * centeringFactor
        boid['dy'] += (centerY - boid['y']) * centeringFactor

def avoidOthers(boid):
    minDistance = 20
    avoidFactor = 0.05
    moveX = 0
    moveY = 0

    for otherBoid in boids:
        if otherBoid != boid:
            if distance(boid, otherBoid) < minDistance:
                moveX += boid['x'] - otherBoid['x']
                moveY += boid['y'] - otherBoid['y']

    boid['dx'] += moveX * avoidFactor
    boid['dy'] += moveY * avoidFactor

def matchVelocity(boid):
    matchingFactor = 0.05

    avgDX = 0
    avgDY = 0
    numNeighbors = 0

    for otherBoid in boids:
        if distance(boid, otherBoid) < visualRange:
            avgDX += otherBoid['dx']
            avgDY += otherBoid['dy']
            numNeighbors += 1

    if numNeighbors:
        avgDX /= numNeighbors
        avgDY /= numNeighbors

        boid['dx'] += (avgDX - boid['dx']) * matchingFactor
        boid['dy'] += (avgDY - boid['dy']) * matchingFactor

def limitSpeed(boid):
    speedLimit = 10

    speed = math.sqrt(boid['dx']**2 + boid['dy']**2)
    if speed > speedLimit:
        boid['dx'] = (boid['dx'] / speed) * speedLimit
        boid['dy'] = (boid['dy'] / speed) * speedLimit

def drawBoid(screen, boid):
    angle = math.atan2(boid['dy'], boid['dx'])
    boid_surface = pygame.Surface((30, 10), pygame.SRCALPHA)
    pygame.draw.polygon(boid_surface, (85, 140, 244), [
        (0, 0), (0, 10), (-15, 5)
    ])
    rotated_boid = pygame.transform.rotate(boid_surface, math.degrees(angle))
    rotated_rect = rotated_boid.get_rect(center=(boid['x'], boid['y']))
    screen.blit(rotated_boid, rotated_rect)

    if DRAW_TRAIL:
        for point in boid['history']:
            pygame.draw.circle(screen, (85, 140, 244, 102), (int(point[0]), int(point[1])), 1)

# Main animation loop
def animationLoop():
    global boids
    for boid in boids:
        flyTowardsCenter(boid)
        avoidOthers(boid)
        matchVelocity(boid)
        limitSpeed(boid)
        keepWithinBounds(boid)

        boid['x'] += boid['dx']
        boid['y'] += boid['dy']
        boid['history'].append((boid['x'], boid['y']))
        boid['history'] = boid['history'][-50:]

    screen.fill((255, 255, 255))
    for boid in boids:
        drawBoid(screen, boid)

    pygame.display.flip()
    pygame.time.Clock().tick(60)

pygame.init()
screen = sizeCanvas()
DRAW_TRAIL = False

initBoids()
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
    animationLoop()

SystemExit: 

In [10]:
boids

[{'x': 335.66378089546805,
  'y': 624.7109894623812,
  'dx': -5.472417329481836,
  'dy': -0.07669865396076322,
  'history': [(642.109134659251, 763.1234790062998),
   (637.0734338490597, 759.8120342938814),
   (631.9671937216156, 756.4237647432691),
   (626.7905023025546, 752.9606710595023),
   (621.5433181477769, 749.4247525778826),
   (616.1946743699024, 745.7941714263045),
   (610.7459408955917, 742.0742844048382),
   (605.1987391518053, 738.2705765655883),
   (599.5540825096238, 734.3884682686823),
   (593.8126487759405, 730.4333407357327),
   (587.9754189162963, 726.4107000354496),
   (582.0432382188862, 722.3260366208738),
   (576.016766236921, 718.184741541463),
   (569.896469065039, 713.9920915513228),
   (563.6837697250737, 709.7532032947269),
   (557.3803545138106, 705.473610150393),
   (550.9878094357193, 701.1587152533185),
   (544.5076160118978, 696.8137726262781),
   (537.9420292781906, 692.4441904946594),
   (531.2938673417976, 688.0554862912185),
   (524.5658879452197, 

In [20]:
boid_json =[]
dc={}
list_of_ 

for i in range(len(boids[1]["history"])): # go through the length of history
    print(i)
    for j in range(len(boids)): # go throuugh the number of boids for every second
        dc[f'boid{j}'] = boids[j]['history'][i] # create a key of boid1,2,3 wtih the (x,y)

boid_json.append(dc)

    

#  boad_saved = boids



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [24]:
time_list=[]



for i in range(len(boids[1]["history"])): # go through the length of history
    for j in range(len(boids)): # go throuugh the number of boids for every second
        boid_dict={}
        boid_dict[f'boid{j}'] = (boids[j]['history'][i][0],boids[j]['history'][i][1],boids[j]['dx'],boids[j]['dy'])
        time_list.append[boid_dict]
    


    

TypeError: 'builtin_function_or_method' object is not subscriptable

In [25]:
time_list = []  # Ensure time_list is initialized

for i in range(len(boids[1]["history"])):  # Go through the length of history
    boid_dict = {}  # Initialize the dictionary for the current time step
    x={}
    y={}
    dx={}
    dy={}
    for j in range(len(boids)):  # Go through the number of boids for every second
        
        boid_dict[f'boid{j}'] = (boids[j]['history'][i][0], boids[j]['history'][i][1], boids[j]['dx'], boids[j]['dy'])
    time_list.append(boid_dict)  # Append the dictionary to the list


In [26]:
time_list

[{'boid0': (642.109134659251,
   763.1234790062998,
   -5.472417329481836,
   -0.07669865396076322),
  'boid1': (392.3113635128244,
   325.708714016338,
   -0.0925541847057502,
   8.372145378982843),
  'boid2': (484.8160588184143,
   357.5543105920764,
   4.4489962689366465,
   7.94957776512397),
  'boid3': (440.88383697328686,
   744.3066276112514,
   7.992924332874066,
   -5.275890499254062),
  'boid4': (350.24413478652326,
   749.9038008391374,
   7.328719791944818,
   -5.062069190800808),
  'boid5': (293.2958097971605,
   783.5154743268259,
   8.028113235217798,
   -3.075577424360276),
  'boid6': (506.65700451333976,
   343.8110469323714,
   3.397594539643403,
   8.794441941358183),
  'boid7': (507.9032507022588,
   666.6802257381897,
   3.94435168186782,
   -3.645554835773062),
  'boid8': (325.02446274692704,
   366.3466315864732,
   -2.2902072035744068,
   5.781836313644899),
  'boid9': (486.9052904846746,
   724.0417547727543,
   5.663555201621524,
   -5.115364504688556),
  'boi

In [29]:
time_list = []  # Ensure time_list is initialized

for i in range(len(boids[1]["history"])):  # Go through the length of history
    boid_dict = {}  # Initialize the dictionary for the current time step
    for j in range(len(boids)):  # Go through the number of boids for every second
        dic = {}  # Initialize the dictionary for the current boid
        dic['x'] = boids[j]['history'][i][0]
        dic['y'] = boids[j]['history'][i][1]
        dic['dx'] = boids[j]['dx']
        dic['dy'] = boids[j]['dy']
        boid_dict[f'boid{j}'] = dic
    time_list.append(boid_dict)  # Append the dictionary to the list

In [30]:
time_list

[{'boid0': {'x': 642.109134659251,
   'y': 763.1234790062998,
   'dx': -5.472417329481836,
   'dy': -0.07669865396076322},
  'boid1': {'x': 392.3113635128244,
   'y': 325.708714016338,
   'dx': -0.0925541847057502,
   'dy': 8.372145378982843},
  'boid2': {'x': 484.8160588184143,
   'y': 357.5543105920764,
   'dx': 4.4489962689366465,
   'dy': 7.94957776512397},
  'boid3': {'x': 440.88383697328686,
   'y': 744.3066276112514,
   'dx': 7.992924332874066,
   'dy': -5.275890499254062},
  'boid4': {'x': 350.24413478652326,
   'y': 749.9038008391374,
   'dx': 7.328719791944818,
   'dy': -5.062069190800808},
  'boid5': {'x': 293.2958097971605,
   'y': 783.5154743268259,
   'dx': 8.028113235217798,
   'dy': -3.075577424360276},
  'boid6': {'x': 506.65700451333976,
   'y': 343.8110469323714,
   'dx': 3.397594539643403,
   'dy': 8.794441941358183},
  'boid7': {'x': 507.9032507022588,
   'y': 666.6802257381897,
   'dx': 3.94435168186782,
   'dy': -3.645554835773062},
  'boid8': {'x': 325.024462746

In [34]:
import json
jake = json.dumps(time_list)

str

In [35]:
with open("mydata.json", "w") as final:
    json.dump(jake, final)
#save it

In [37]:
with open('mydata.json') as json_file:
    data = json.load(json_file)
    print(data)
#load it

[{"boid0": {"x": 642.109134659251, "y": 763.1234790062998, "dx": -5.472417329481836, "dy": -0.07669865396076322}, "boid1": {"x": 392.3113635128244, "y": 325.708714016338, "dx": -0.0925541847057502, "dy": 8.372145378982843}, "boid2": {"x": 484.8160588184143, "y": 357.5543105920764, "dx": 4.4489962689366465, "dy": 7.94957776512397}, "boid3": {"x": 440.88383697328686, "y": 744.3066276112514, "dx": 7.992924332874066, "dy": -5.275890499254062}, "boid4": {"x": 350.24413478652326, "y": 749.9038008391374, "dx": 7.328719791944818, "dy": -5.062069190800808}, "boid5": {"x": 293.2958097971605, "y": 783.5154743268259, "dx": 8.028113235217798, "dy": -3.075577424360276}, "boid6": {"x": 506.65700451333976, "y": 343.8110469323714, "dx": 3.397594539643403, "dy": 8.794441941358183}, "boid7": {"x": 507.9032507022588, "y": 666.6802257381897, "dx": 3.94435168186782, "dy": -3.645554835773062}, "boid8": {"x": 325.02446274692704, "y": 366.3466315864732, "dx": -2.2902072035744068, "dy": 5.781836313644899}, "boi